In [1]:
# Download Mask RCNN Repository
! git clone https://github.com/matterport/Mask_RCNN.git

fatal: destination path 'Mask_RCNN' already exists and is not an empty directory.


In [1]:
# installing dependency for google collab
!pip uninstall -y tensorflow
!pip uninstall -y tensorflow-gpu
!pip install tensorflow-gpu==1.14.0
!pip install keras==2.1.3

  Using cached https://files.pythonhosted.org/packages/76/04/43153bfdfcf6c9a4c38ecdb971ca9a75b9a791bb69a764d652c359aca504/tensorflow_gpu-1.14.0-cp36-cp36m-manylinux1_x86_64.whl


In [3]:
from keras import backend as K
import tensorflow as tf

# silent tf verbose
tf.get_logger().setLevel('INFO')
# check to see if gpu is used
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [4]:
import os
import sys
import json
from imgaug import augmenters as iaa
import numpy as np

In [5]:
# Root directory of the project
# When drive is mounted, prepend content 
ROOT_DIR = os.path.join('Mask_RCNN')
sys.path.append(ROOT_DIR)
# append mounted google drive to path for using mrcnn
GDRIVE_DATASET_PATH  = os.path.join('drive','My Drive','Colab Notebooks','mask_rcnn')
sys.path.append(GDRIVE_DATASET_PATH)

In [6]:
# Global variables after path is mounted
# path/gdrive path where the model is saved
MODEL_PERS_DIR = os.path.join(GDRIVE_DATASET_PATH, 'farm_dam', 'model')
# training part
DATASET_PATH = os.path.join(os.getcwd(), GDRIVE_DATASET_PATH,'farm_dam')
# google drive to coco model
COCO_MODEL_PATH = os.path.join(GDRIVE_DATASET_PATH, "mask_rcnn_coco.h5")

In [7]:
# loading modules after path is added
from loader import DamConfig, FarmDamDataset
import mrcnn.model as modellib

In [16]:
# This code section is added for debugging, since needed to upload loader each time
from mrcnn.config import Config

class DamConfig(Config):
    """
    Configuration for training on the dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "farm_dam"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + FarmDam Class

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 1024
    IMAGE_MAX_DIM = 1024
    
    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (16, 32, 64, 128, 256)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 16

    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.7


    # How many anchors per image to use for RPN training
    RPN_TRAIN_ANCHORS_PER_IMAGE = 1024

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100

    # Skip detections with < 80% confidence
    DETECTION_MIN_CONFIDENCE = 0.7

    WEIGHT_DECAY = 0.001
    LEARNING_RATE = 0.001
    VALIDATION_STEPS = 10

In [17]:
config = DamConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  1024
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE        

In [18]:
# training images dir
dataset_train = FarmDamDataset()
dataset_train.load_dam(DATASET_PATH, "train")
dataset_train.prepare()

# validation images dir
dataset_val = FarmDamDataset()
dataset_val.load_dam(DATASET_PATH, "val")
dataset_val.prepare()

In [19]:
# Initializing augmentation
# augmentation = iaa.SomeOf((0, 5), [
#     iaa.Fliplr(0.5),
#     iaa.Flipud(0.4),
#     iaa.Crop(percent=(0, 0.5)), # random crops,
#     iaa.Dropout([0.05, 0.2]),      # drop 5% or 20% of all pixels
#     iaa.SomeOf((0, 4),[iaa.Affine(rotate=45),
#                 iaa.Affine(rotate=90),
#                 iaa.Affine(rotate=135),
#                 iaa.Affine(rotate=180),
#                 iaa.Affine(rotate=270)]),
#     # Change brightness of images (80-150% of original value).
#     iaa.Multiply((0.8, 1.5)),
#     iaa.GaussianBlur(sigma=(0.0, 5.0))
    
# ])

augmentation = iaa.SomeOf((0, 5), [
    iaa.Fliplr(0.5),
    iaa.Flipud(0.4),
    iaa.SomeOf((0, 4),[iaa.Affine(rotate=45),
                iaa.Affine(rotate=90),
                iaa.Affine(rotate=135),
                iaa.Affine(rotate=180),
                iaa.Affine(rotate=270)]),
    # Change brightness of images (80-150% of original value).
    iaa.Multiply((0.8, 1.5)),
])

In [20]:
def get_pretrained_model(pretrained_model_weights, model_directory, train_from_last_saved_epoch = False):
  # Create model in training mode
  model = modellib.MaskRCNN(mode="training", config=config, model_dir=model_directory)
  # Load weights trained on MS COCO, but skip layers that
  # are different due to the different number of classes
  
  if not train_from_last_saved_epoch:
    # Transfer learning from start
    model.load_weights(pretrained_model_weights, by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc","mrcnn_bbox", "mrcnn_mask"])
  else:
    # train from last saved epoch
    weights_path = model.find_last()
    model.load_weights(weights_path, by_name=True)
  return model

In [21]:
# Global variable
# If model training is to be restarted from last saved epoch
train_from_last_saved_epoch = False
EPOCHS=40
# get the model initialized in Coco dataset
model = get_pretrained_model(COCO_MODEL_PATH, MODEL_PERS_DIR, train_from_last_saved_epoch)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


In [ ]:
# Train the head branches.
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=EPOCHS,
            augmentation=augmentation,
            layers='heads')


Starting at epoch 0. LR=0.001

Checkpoint Path: drive/My Drive/Colab Notebooks/mask_rcnn/farm_dam/model/farm_dam20201005T0942/mask_rcnn_farm_dam_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4  

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:2033: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may du



Epoch 1/40
100/100 [==============================] - 286s 3s/step - loss: 2.1850 - rpn_class_loss: 0.0096 - rpn_bbox_loss: 0.8070 - mrcnn_class_loss: 0.2910 - mrcnn_bbox_loss: 0.5266 - mrcnn_mask_loss: 0.5508 - val_loss: 1.5805 - val_rpn_class_loss: 0.0130 - val_rpn_bbox_loss: 0.4915 - val_mrcnn_class_loss: 0.2957 - val_mrcnn_bbox_loss: 0.4356 - val_mrcnn_mask_loss: 0.3447
Epoch 2/40
100/100 [==============================] - 119s 1s/step - loss: 1.4021 - rpn_class_loss: 0.0084 - rpn_bbox_loss: 0.5054 - mrcnn_class_loss: 0.2196 - mrcnn_bbox_loss: 0.3831 - mrcnn_mask_loss: 0.2856 - val_loss: 1.3679 - val_rpn_class_loss: 0.0130 - val_rpn_bbox_loss: 0.4363 - val_mrcnn_class_loss: 0.2475 - val_mrcnn_bbox_loss: 0.3605 - val_mrcnn_mask_loss: 0.3106
Epoch 3/40
100/100 [==============================] - 124s 1s/step - loss: 1.2310 - rpn_class_loss: 0.0090 - rpn_bbox_loss: 0.4854 - mrcnn_class_loss: 0.1797 - mrcnn_bbox_loss: 0.2994 - mrcnn_mask_loss: 0.2575 - val_loss: 1.2663 - val_rpn_class

In [ ]:
print("Train all layers")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=125,
            augmentation=augmentation,
            layers='all')

Train all layers

Starting at epoch 110. LR=1e-05

Checkpoint Path: drive/My Drive/Colab Notebooks/mask_rcnn/farm_dam/model/farm_dam20201003T1432/mask_rcnn_farm_dam_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (B

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:2033: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may du

Epoch 111/125
20/20 [==============================] - 594s 30s/step - loss: 2.1760 - rpn_class_loss: 0.0377 - rpn_bbox_loss: 1.7014 - mrcnn_class_loss: 0.0098 - mrcnn_bbox_loss: 0.2066 - mrcnn_mask_loss: 0.2204 - val_loss: 2.4016 - val_rpn_class_loss: 0.1269 - val_rpn_bbox_loss: 1.5225 - val_mrcnn_class_loss: 0.0257 - val_mrcnn_bbox_loss: 0.3545 - val_mrcnn_mask_loss: 0.3719
Epoch 112/125
20/20 [==============================] - 44s 2s/step - loss: 2.0934 - rpn_class_loss: 0.0490 - rpn_bbox_loss: 1.5096 - mrcnn_class_loss: 0.0102 - mrcnn_bbox_loss: 0.2023 - mrcnn_mask_loss: 0.3223 - val_loss: 2.4401 - val_rpn_class_loss: 0.1291 - val_rpn_bbox_loss: 1.5181 - val_mrcnn_class_loss: 0.0258 - val_mrcnn_bbox_loss: 0.3781 - val_mrcnn_mask_loss: 0.3890
Epoch 113/125
20/20 [==============================] - 49s 2s/step - loss: 2.3265 - rpn_class_loss: 0.0403 - rpn_bbox_loss: 1.8153 - mrcnn_class_loss: 0.0067 - mrcnn_bbox_loss: 0.2162 - mrcnn_mask_loss: 0.2479 - val_loss: 2.4116 - val_rpn_class